In [2]:
import random

examples = 10000
symbols = 100
length = 10

with open('vocab', 'w') as f:
    f.write("<S>\n</S>\n<UNK>\n")
    for i in range(symbols):
        f.write("%d\n" % i)


with open('input', 'w') as fin:
    with open('output', 'w') as fout:
        for i in range(examples):
            inp = [random.randint(0, symbols) + 3 for _ in range(length)]
            out = [(x + 5) % 100 + 3 for x in inp]
            fin.write(' '.join([str(x) for x in inp]) + '\n')
            fout.write(' '.join([str(x) for x in out]) + '\n')


In [1]:
import logging

import numpy as np
import tensorflow as tf
from tensorflow.contrib import layers

import timeline


GO_TOKEN = 0
END_TOKEN = 1
UNK_TOKEN = 2


def seq2seq(mode, features, labels, params):
    vocab_size = params['vocab_size']
    embed_dim = params['embed_dim']
    num_units = params['num_units']
    input_max_length = params['input_max_length']
    output_max_length = params['output_max_length']

    inp = features['input']
    output = features['output']
    batch_size = tf.shape(inp)[0]
    start_tokens = tf.zeros([batch_size], dtype=tf.int64)
    train_output = tf.concat([tf.expand_dims(start_tokens, 1), output], 1)
    input_lengths = tf.reduce_sum(tf.to_int32(tf.not_equal(inp, 1)), 1)
    output_lengths = tf.reduce_sum(tf.to_int32(tf.not_equal(train_output, 1)), 1)
    input_embed = layers.embed_sequence(
        inp, vocab_size=vocab_size, embed_dim=embed_dim, scope='embed')
    output_embed = layers.embed_sequence(
        train_output, vocab_size=vocab_size, embed_dim=embed_dim, scope='embed', reuse=True)
    with tf.variable_scope('embed', reuse=True):
        embeddings = tf.get_variable('embeddings')

    cell = tf.contrib.rnn.GRUCell(num_units=num_units)
    encoder_outputs, encoder_final_state = tf.nn.dynamic_rnn(cell, input_embed, dtype=tf.float32)

    train_helper = tf.contrib.seq2seq.TrainingHelper(output_embed, output_lengths)
    # train_helper = tf.contrib.seq2seq.ScheduledEmbeddingTrainingHelper(
    #     output_embed, output_lengths, embeddings, 0.3
    # )
    pred_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
        embeddings, start_tokens=tf.to_int32(start_tokens), end_token=1)

    def decode(helper, scope, reuse=None):
        with tf.variable_scope(scope, reuse=reuse):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
                num_units=num_units, memory=encoder_outputs,
                memory_sequence_length=input_lengths)
            cell = tf.contrib.rnn.GRUCell(num_units=num_units)
            attn_cell = tf.contrib.seq2seq.AttentionWrapper(
                cell, attention_mechanism, attention_layer_size=num_units / 2)
            out_cell = tf.contrib.rnn.OutputProjectionWrapper(
                attn_cell, vocab_size, reuse=reuse
            )
            decoder = tf.contrib.seq2seq.BasicDecoder(
                cell=out_cell, helper=helper,
                initial_state=out_cell.zero_state(
                    dtype=tf.float32, batch_size=batch_size))
                #initial_state=encoder_final_state)
            outputs = tf.contrib.seq2seq.dynamic_decode(
                decoder=decoder, output_time_major=False,
                impute_finished=True, maximum_iterations=output_max_length
            )
            return outputs[0]
    train_outputs = decode(train_helper, 'decode')
    pred_outputs = decode(pred_helper, 'decode', reuse=True)

    tf.identity(train_outputs.sample_id[0], name='train_pred')
    weights = tf.to_float(tf.not_equal(train_output[:, :-1], 1))
    loss = tf.contrib.seq2seq.sequence_loss(
        train_outputs.rnn_output, output, weights=weights)
    train_op = layers.optimize_loss(
        loss, tf.train.get_global_step(),
        optimizer=params.get('optimizer', 'Adam'),
        learning_rate=params.get('learning_rate', 0.001),
        summaries=['loss', 'learning_rate'])

    tf.identity(pred_outputs.sample_id[0], name='predictions')
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=pred_outputs.sample_id,
        loss=loss,
        train_op=train_op
    )


def tokenize_and_map(line, vocab):
    return [vocab.get(token, UNK_TOKEN) for token in line.split(' ')]


def make_input_fn(
        batch_size, input_filename, output_filename, vocab,
        input_max_length, output_max_length,
        input_process=tokenize_and_map, output_process=tokenize_and_map):

    def input_fn():
        inp = tf.placeholder(tf.int64, shape=[None, None], name='input')
        output = tf.placeholder(tf.int64, shape=[None, None], name='output')
        tf.identity(inp[0], 'input_0')
        tf.identity(output[0], 'output_0')
        return {
            'input': inp,
            'output': output,
        }, None

    def sampler():
        while True:
            with open(input_filename) as finput:
                with open(output_filename) as foutput:
                    for in_line in finput:
                        out_line = foutput.readline()
                        yield {
                            'input': input_process(in_line, vocab)[:input_max_length - 1] + [END_TOKEN],
                            'output': output_process(out_line, vocab)[:output_max_length - 1] + [END_TOKEN]
                        }

    sample_me = sampler()

    def feed_fn():
        inputs, outputs = [], []
        input_length, output_length = 0, 0
        for i in range(batch_size):
            rec = next(sample_me)
            inputs.append(rec['input'])
            outputs.append(rec['output'])
            input_length = max(input_length, len(inputs[-1]))
            output_length = max(output_length, len(outputs[-1]))
        # Pad me right with </S> token.
        for i in range(batch_size):
            inputs[i] += [END_TOKEN] * (input_length - len(inputs[i]))
            outputs[i] += [END_TOKEN] * (output_length - len(outputs[i]))
        return {
            'input:0': inputs,
            'output:0': outputs
        }

    return input_fn, feed_fn


def load_vocab(filename):
    vocab = {}
    with open(filename) as f:
        for idx, line in enumerate(f):
            vocab[line.strip()] = idx
    return vocab


def get_rev_vocab(vocab):
    return {idx: key for key, idx in vocab.items()}


def get_formatter(keys, vocab):
    rev_vocab = get_rev_vocab(vocab)

    def to_str(sequence):
        tokens = [
            rev_vocab.get(x, "<UNK>") for x in sequence]
        return ' '.join(tokens)

    def format(values):
        res = []
        for key in keys:
            res.append("%s = %s" % (key, to_str(values[key])))
        return '\n'.join(res)
    return format


def train_seq2seq(
        input_filename, output_filename, vocab_filename,
        model_dir):
    vocab = load_vocab(vocab_filename)
    params = {
        'vocab_size': len(vocab),
        'batch_size': 32,
        'input_max_length': 30,
        'output_max_length': 30,
        'embed_dim': 100,
        'num_units': 256
    }
    est = tf.estimator.Estimator(
        model_fn=seq2seq,
        model_dir=model_dir, 
        params=params)

    input_fn, feed_fn = make_input_fn(
        params['batch_size'],
        input_filename,
        output_filename,
        vocab, 
        params['input_max_length'], 
        params['output_max_length'])

    # Make hooks to print examples of inputs/predictions.
    print_inputs = tf.train.LoggingTensorHook(
        ['input_0', 'output_0'], every_n_iter=100,
        formatter=get_formatter(['input_0', 'output_0'], vocab))
    print_predictions = tf.train.LoggingTensorHook(
        ['predictions', 'train_pred'], every_n_iter=100,
        formatter=get_formatter(['predictions', 'train_pred'], vocab))

    timeline_hook = timeline.TimelineHook(model_dir, every_n_iter=100)
    est.train(
        input_fn=input_fn,
        hooks=[tf.train.FeedFnHook(feed_fn), print_inputs, print_predictions, timeline_hook],
        steps=10000)


def main():
    tf.logging._logger.setLevel(logging.INFO)
    train_seq2seq('input', 'output', 'vocab', 'model/seq2seq')


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        pass
        


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa34387c9e8>, '_is_chief': True, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_session_config': None, '_save_summary_steps': 100, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_save_checkpoints_steps': None, '_model_dir': 'model/seq2seq', '_save_checkpoints_secs': 600, '_service': None, '_master': '', '_tf_random_seed': None, '_task_id': 0, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into model/seq2seq/model.ckpt.
INFO:tensorflow:input_0 = 29 67 43 37 48 77 17 74 40 <UNK> </S>
output_0 = 37 75 51 45 56 85 25 82 48 <UNK> </S>
INFO:tensorflow:predictions = 61 64 64 44 84 60 2 79 79 79 79 79 79 47 58 47 83 83 83 17 17 2 36 36 36 36 36 77 94 94
train_pred = 61 9 61 60 84 60 53 53 79 60 60
INFO:tensorflow:loss = 4.63824, 

INFO:tensorflow:loss = 0.0629475, step = 2201 (13.557 sec)
INFO:tensorflow:global_step/sec: 7.39367
INFO:tensorflow:input_0 = 41 55 42 91 92 75 48 56 61 <UNK> </S>
output_0 = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
INFO:tensorflow:predictions = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
train_pred = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
INFO:tensorflow:loss = 0.0376037, step = 2301 (13.069 sec)
INFO:tensorflow:global_step/sec: 7.50112
INFO:tensorflow:input_0 = 9 86 34 37 73 97 56 5 93 <UNK> </S>
output_0 = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S>
INFO:tensorflow:predictions = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S>
train_pred = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S>
INFO:tensorflow:loss = 0.0323325, step = 2401 (13.426 sec)
INFO:tensorflow:global_step/sec: 7.68949
INFO:tensorflow:input_0 = 29 67 43 37 48 77 17 74 40 <UNK> </S>
output_0 = 37 75 51 45 56 85 25 82 48 <UNK> </S>
INFO:tensorflow:predictions = 37 75 51 45 56 85 25 82 48 <UNK> </S>
train_pred = 37 75 51 45 56 85 25 82 48 <U

INFO:tensorflow:predictions = 74 92 58 64 10 48 80 40 99 <UNK> </S>
train_pred = 74 92 58 64 10 48 80 40 99 <UNK> </S>
INFO:tensorflow:loss = 0.0572809, step = 4701 (12.473 sec)
INFO:tensorflow:Saving checkpoints for 4702 into model/seq2seq/model.ckpt.
INFO:tensorflow:global_step/sec: 5.94281
INFO:tensorflow:input_0 = 41 55 42 91 92 75 48 56 61 <UNK> </S>
output_0 = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
INFO:tensorflow:predictions = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
train_pred = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
INFO:tensorflow:loss = 0.0312285, step = 4801 (16.527 sec)
INFO:tensorflow:global_step/sec: 7.4205
INFO:tensorflow:input_0 = 9 86 34 37 73 97 56 5 93 <UNK> </S>
output_0 = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S>
INFO:tensorflow:predictions = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S>
train_pred = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S>
INFO:tensorflow:loss = 0.0419472, step = 4901 (13.469 sec)
INFO:tensorflow:global_step/sec: 7.54586
INFO:tensorflow:input_0 = 29 6

INFO:tensorflow:global_step/sec: 8.13851
INFO:tensorflow:input_0 = 66 84 50 56 <UNK> 40 72 32 91 <UNK> </S>
output_0 = 74 92 58 64 8 48 80 40 99 <UNK> </S>
INFO:tensorflow:predictions = 74 92 58 64 10 48 80 40 99 <UNK> </S>
train_pred = 74 92 58 64 10 48 80 40 99 <UNK> </S>
INFO:tensorflow:loss = 0.0567328, step = 7201 (12.552 sec)
INFO:tensorflow:global_step/sec: 7.72059
INFO:tensorflow:input_0 = 41 55 42 91 92 75 48 56 61 <UNK> </S>
output_0 = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
INFO:tensorflow:predictions = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
train_pred = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
INFO:tensorflow:loss = 0.0264313, step = 7301 (13.108 sec)
INFO:tensorflow:global_step/sec: 7.2769
INFO:tensorflow:input_0 = 9 86 34 37 73 97 56 5 93 <UNK> </S>
output_0 = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S>
INFO:tensorflow:predictions = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S>
train_pred = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S>
INFO:tensorflow:loss = 0.0308675, step = 7401 (13

INFO:tensorflow:predictions = 89 55 11 55 73 45 66 6 28 <UNK> </S>
train_pred = 89 55 11 55 73 45 66 6 28 <UNK> </S>
INFO:tensorflow:loss = 0.0305936, step = 9601 (12.908 sec)
INFO:tensorflow:global_step/sec: 7.09413
INFO:tensorflow:input_0 = 66 84 50 56 <UNK> 40 72 32 91 <UNK> </S>
output_0 = 74 92 58 64 8 48 80 40 99 <UNK> </S>
INFO:tensorflow:predictions = 74 92 58 64 8 48 80 40 99 <UNK> </S>
train_pred = 74 92 58 64 8 48 80 40 99 <UNK> </S>
INFO:tensorflow:loss = 0.0331468, step = 9701 (14.240 sec)
INFO:tensorflow:global_step/sec: 7.21251
INFO:tensorflow:input_0 = 41 55 42 91 92 75 48 56 61 <UNK> </S>
output_0 = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
INFO:tensorflow:predictions = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
train_pred = 49 63 50 99 <UNK> 83 56 64 69 <UNK> </S>
INFO:tensorflow:loss = 0.0286003, step = 9801 (13.836 sec)
INFO:tensorflow:global_step/sec: 7.5948
INFO:tensorflow:input_0 = 9 86 34 37 73 97 56 5 93 <UNK> </S>
output_0 = 17 94 42 45 81 5 64 13 <UNK> <UNK> </S